# Imports

In [79]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import seaborn as sns
from sklearn import svm, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

# Read Data

In [80]:
df = pd.DataFrame(np.load(r'../data/master_prepared.npy', allow_pickle=True))
cols = np.load(r'../data/columns_prepared.npy', allow_pickle=True)
df.columns = cols

In [115]:
# Create polynomials

df['mv_^0.5'] = df['mv'] ** 0.5

df['bvtmv_^2'] = df['bvtmv'] ** 2
df['bvtmv_^3'] = df['bvtmv'] ** 3


In [124]:
drop_cols = [
    'date',
    'eligible',
    'companyid', 
    'ret_3m',
    'ret_6m',
    'std_3m',
    'std_6m',
    'map_6m',
    'r_1f1',
    'r_1f3',
    'r_1f6',
    'r_1f12'
]
dates = df['date'].unique()

# Regress on next periods return
y_col = 'r_1f1'

# Independent variables
x_cols = [
    'bvtmv', 
    'bvtmv_^2', 
    'bvtmv_^3', 
    'ret_12m', 
    'std_12m', 
    'fcf', 
    'mv', 
    'mv_^0.5', 
    'roic', 
    'dy', 
    'beta', 
    'so',
]
# x_cols = [i for i in list(df.columns) if i not in drop_cols]

df_eligible = df[df['eligible']==1]
X = df_eligible[x_cols].astype('float')
y = df_eligible[y_col].astype('float')


In [127]:
# Start index of dates, and index range to regress over
start_index = 1
train_range = 12

train_dates = dates[start_index:start_index+train_range+1]
test_dates = dates[start_index+train_range+1]

X_train = X[df_eligible['date'].isin(train_dates)].astype('float')
X_train = sm.add_constant(X_train)
y_train = y[df_eligible['date'].isin(train_dates)].astype('float')

X_test = X[df_eligible['date']==test_dates].astype('float')
X_test.insert(0, 'const', 1)
y_test = y[df_eligible['date']==test_dates].astype('float')

# print('Train Range:  ', train_dates)
# print('Test Range:  ', test_dates)
print('X_train: ', X_train.values.shape)
print('y_train: ', y_train.values.shape)
print('X_test:  ', X_test.values.shape)
print('y_test:  ', y_test.values.shape)

# X_train = preprocessing.scale(X_train)
# X_test = preprocessing.scale(X_test)

X_train:  (1682, 13)
y_train:  (1682,)
X_test:   (837, 13)
y_test:   (837,)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


# Model

In [128]:
model = sm.OLS(y_train, X_train).fit()
pred = model.predict(X_test)

# pred.plot()
# plt.show()
print(model.summary())
# print(model.params)
print('R2_oos:       ', r2_score(y_test, pred))
print('R2_is:        ', model.rsquared)

                            OLS Regression Results                            
Dep. Variable:                  r_1f1   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     4.108
Date:                Wed, 05 Feb 2020   Prob (F-statistic):           2.33e-06
Time:                        13:59:48   Log-Likelihood:                 1146.1
No. Observations:                1682   AIC:                            -2266.
Df Residuals:                    1669   BIC:                            -2196.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0178      0.013      1.412      0.1